# Program 3: ID3 algorithm

## import math and csv

In [78]:
import math
import csv

## separate header and remaining instances from the dataset

In [79]:
def loadCsv(filename):

    lines=csv.reader(open(filename,"r"));

    dataset = list(lines)

    headers = dataset.pop(0)

    return dataset,headers

## Node class used to build a tree

In [80]:
class Node:

    def __init__(self,attribute):

        self.attribute=attribute

        self.children=[]

        self.answer=" "

In [81]:
def subtables(data,col,delete):

    dic={}

    coldata=[row[col]for row in data]

    attr=list(set(coldata))

    

    counts=[0]*len(attr)

    r=len(data)

    c=len(data[0])

    for x in range (len(attr)):

        for y in range(r):

            if data[y][col]==attr[x]:

                counts[x]+=1

    for x in range(len(attr)):

        dic[attr[x]]=[[0 for i in range (c)]for j in range(counts[x])]

        pos=0

        for y in range(r):

            if data[y][col]==attr[x]:

                if delete:

                    del data[y][col]

                dic[attr[x]][pos]=data[y]

                pos+=1

    return attr,dic

## Entropy function: uncertainity in dataset

In [82]:
def entropy(S):

    attr = list(set(S))

    if len(attr)==1:

        return 0

    counts=[0,0]

    for i in range(2):

        counts[i]=sum([1 for x in S if attr[i]==x])/(len(S)*1.0)

    sums=0

    for cnt in counts:

        sums+=-1 * cnt * math.log(cnt,2)

    return sums

## to calculate the highest gain

In [83]:
def compute_gain(data,col):

    attr,dic = subtables(data,col,delete=False)

    total_size = len(data)

    entropies = [0]*len(attr)

    ratio = [0]*len(attr)

    

    total_entropy = entropy([row[-1]for row in data])

    for x in range(len(attr)):

        ratio[x]=len(dic[attr[x]])/(total_size*1.0)

        entropies[x]=entropy([row[-1] for row in dic[attr[x]]])

    for x in range(len(entropies)):

        total_entropy-=ratio[x]*entropies[x]

    return total_entropy 

## to create root and subsequent nodes

In [84]:
def build_tree(data,features):

    lastcol=[row[-1]for row in data]

    if(len(set(lastcol)))==1:

        node=Node("")

        node.answer = lastcol[0]

        return node

    n=len(data[0])-1



    gains=[0]*n

    for col in range(n):

        gains[col]=compute_gain(data,col)

    split=gains.index(max(gains))

    node=Node(features[split])

    fea=features[:split]+features[split+1:]

    attr,dic=subtables(data,split,delete=True)

    for x in range(len(attr)):

        child=build_tree(dic[attr[x]],fea)

        node.children.append((attr[x],child))

    return node

## to print the tree

In [85]:
def print_tree(node,level):

    if node.answer!= " ":

        print(" " * level,node.answer)

        return

    print(" "*level,node.attribute)

    for value,n in node.children:

        print(" "*(level+1),value)

        print_tree(n,level+2)

In [86]:
def classify(node,x_test,features):

    if node.answer != " ":

        print(node.answer)

        return

    pos=features.index(node.attribute)

    for value,n in node.children:

        if x_test[pos]==value:

           classify(n,x_test,features)

## main function

In [87]:
dataset,features=loadCsv("ID3Data.csv")

print(features)
node=build_tree(dataset,features)
print("the decision tree for the dataset using ID3 algorithm is")
print_tree(node,0)
testdata,features=loadCsv("ID3Data.csv")
for xtest in testdata:
    print("the test instance:",xtest)
    print("the predicted label:",end="")
    classify(node,xtest,features)

['Outlook ', 'temperature', 'Humidity', 'windy', 'Play']
the decision tree for the dataset using ID3 algorithm is
 Outlook 
  Sunny
   Humidity
    Normal
     Yes
    High
     No
  Overcast
   Yes
  Rainy
   windy
    True 
     No
    False 
     Yes
the test instance: ['Sunny', 'Hot', 'High', 'False ', 'No']
the predicted label:No
the test instance: ['Sunny', 'Hot', 'High', 'True ', 'No']
the predicted label:No
the test instance: ['Overcast', 'Hot', 'High', 'False ', 'Yes']
the predicted label:Yes
the test instance: ['Rainy', 'Mild', 'High', 'False ', 'Yes']
the predicted label:Yes
the test instance: ['Rainy', 'Cool ', 'Normal', 'False ', 'Yes']
the predicted label:Yes
the test instance: ['Rainy', 'Cool', 'Normal', 'True ', 'No']
the predicted label:No
the test instance: ['Overcast', 'Cool', 'Normal', 'True ', 'Yes']
the predicted label:Yes
the test instance: ['Sunny', 'Mild', 'High', 'False ', 'No']
the predicted label:No
the test instance: ['Sunny', 'Cool', 'Normal', 'False ', 'Y